#### **1. Mật mã Paillier Cryptosystem**
##### ***1.1. Tạo Khóa (Key Generation)***

Đầu tiên, ta tạo hai số nguyên tố ngẫu nhiên `p` và `q` sao cho tích của chúng `n = p * q` tạo thành một mô-đun RSA hợp lệ.

- Giá trị `n` cần có độ dài bit tương ứng với tiêu chuẩn bảo mật RSA hiện tại (tính đến tháng 1 năm 2022 là 2048 bit).
- Từ hai giá trị nguyên tố `p` và `q`, ta tính giá trị `λ = lcm(p − 1, q − 1)` với `lcm` là bội số chung nhỏ nhất.
- Khóa công khai là mô-đun `n`, còn khóa bí mật là giá trị `λ`.

##### ***1.2. Mã hóa (Encryption)***

Giả sử `m` là bản rõ cần mã hóa, với `m ∈ ℤₙ`.

1. Mỗi lần mã hóa, chọn một phần tử ngẫu nhiên `r ∈ ℤₙ`.
2. Tính bản mã `c` theo công thức:
$$ c = (n + 1)^m \cdot r^n \mod n^2 $$

##### ***1.3. Giải mã (Decryption)***

Với khóa bí mật `λ` và bản mã $c \in \mathbb{Z}_{n^2}^{*}$, thực hiện:

1. Tính: $S = c^\lambda \mod n^2$

2. Sau đó, tính: $T = \varphi(n)^{-1} \mod n$ (Với $\varphi(n)$ là hàm Euler.)

3. Cuối cùng, bản rõ được giải mã bằng công thức:
$$
m = D(c) = \frac{S - 1}{n} \cdot T \mod n
$$

##### ***1.4. Tính chất đồng hình (Homomorphic Property)***

Để chứng minh tính đồng hình của hệ mật Paillier, ta đặt $g = n + 1$. Với hai bản rõ `m₁`, `m₂` và hai giá trị ngẫu nhiên `r₁`, `r₂`, ta có hai bản mã tương ứng:

$$
c_1 = g^{m_1} \cdot r_1^n \mod n^2
$$

$$
c_2 = g^{m_2} \cdot r_2^n \mod n^2
$$

Xét tích của hai bản mã:

$$
c_3 = c_1 \cdot c_2 = (g^{m_1} \cdot r_1^n) \cdot (g^{m_2} \cdot r_2^n) \mod n^2
$$

$$
= g^{m_1 + m_2} \cdot (r_1 \cdot r_2)^n \mod n^2 = E(m_1 + m_2, r_1 \cdot r_2)
$$

→ Tức là: **tích của hai bản mã chính là bản mã của tổng hai bản rõ**.

Do đó, hàm giải mã có tính đồng hình:

$$
D(E(m_1, r_1) \cdot E(m_2, r_2)) = D(E(m_1 + m_2, r_1 r_2)) = m_1 + m_2
$$

Hay nói cách khác:

$$
D(E(m_1, r_1)) + D(E(m_2, r_2)) = D(E(m_1 + m_2, r_1 r_2))
$$


#### **2. Bài toán**
- Đầu tiên, cho `n`, sau đó, yêu cầu đăng nhập hoặc đăng ký
- TH đăng ký:
    + Nếu đăng ký admin, không cho phép, thoát
    + Ngược lại cho phép và username sẽ được thêm `user={username}` để tạo `mac`, in ra `(msg + b'|' + mac).hex()`
- TH đăng nhập:
    + Yêu cầu nhập token, dạng: `(msg|mac).hex()`
    + Sau đó thực hiện xác thực, nếu là admin thì in ra FLAG, ngược lại thì không.

MAC: Message Authentication Code

In [1]:
#CODE bài toán

from Crypto.Util.number import getPrime, bytes_to_long, long_to_bytes
from hashlib import sha256
from secrets import randbits
import os

FLAG = os.getenv('FLAG', 'DUCTF{FLAG_TODO}')

class TokenService: 
    def __init__(self):
        self.p = getPrime(512)          # Tạo p
        self.q = getPrime(512)          # Tạo q
        self.n = self.p * self.q
        self.n2 = self.n * self.n       # Tính n^2
        self.l = (self.p - 1) * (self.q - 1)
        self.g = self.n + 1
        self.mu = pow(self.l, -1, self.n)   # Tính muy
        self.secret = os.urandom(16)        # 0 đến 255

    def _encrypt(self, m):
        r = randbits(1024)
        c = pow(self.g, m, self.n2) * pow(r, self.n, self.n2) % self.n2
        return c

    def _decrypt(self, c):
        return ((pow(c, self.l, self.n2) - 1) // self.n) * self.mu % self.n

    def generate(self, msg):
        '''
        Băm SHA256 của secret và msg, chuyển sang số,
        Sau đó mã hóa giá trị băm này.
        '''
        h = bytes_to_long(sha256(self.secret + msg).digest())
        return long_to_bytes(self._encrypt(h))

    def verify(self, msg, mac):
        '''
        Băm SHA256 của secret và msg, chuyển sang số,
        Sau đó giải mã giá trị mac,
        So sánh w[-32:] (32 byte cuối) với hash h.
        '''
        h = sha256(self.secret + msg).digest()
        w = long_to_bytes(self._decrypt(bytes_to_long(mac)))
        return h == w[-32:]


def menu():
    print('1. Register')
    print('2. Login')
    return int(input('> '))


def main():
    ts = TokenService()
    print(ts.n)

    while True:
        choice = menu()
        if choice == 1:
            username = input('Username: ').encode()
            if b'admin' in username:
                print('Cannot register admin user')
                exit(1)
            msg = b'user=' + username
            mac = ts.generate(msg)
            print('Token:', (msg + b'|' + mac).hex())
        elif choice == 2:
            token = bytes.fromhex(input('Token: '))
            msg, _, mac = token.partition(b'|')
            if ts.verify(msg, mac):
                user = msg.rpartition(b'user=')[2]
                print(f'Welcome {user}!')
                if user == b'admin':
                    print(FLAG)
            else:
                print('Failed to verify token')
        else:
            exit(1)
# if __name__ == '__main__':
#     main()

In [2]:
from Crypto.Util.number import getPrime, bytes_to_long, long_to_bytes
from hashlib import sha256
from secrets import randbits
import os

from pwn import *

In [ ]:
ts = TokenService()
secret = ts.secret
n = 62193965648465399096623579181477183233254274930926626512971528815428012636286805522802093361594855935320197783306741361574261363816402418047847456810197277141794374405377051042177440809964553006570936846928894075273768966204852427590549910876763841563072715373847778452274966968494291019990427851608556125393
n2 = n * n
g = n + 1
r = randbits(1024)
username = b'ae'
msg = b'user=' + username

# ============================== REGISTER ==============================
# mac = ts.generate(msg)
hash = bytes_to_long(sha256(secret + msg).digest())
# mac = long_to_bytes(pow(g, hash, n2) * pow(r, n, n2) % n2)
mac = long_to_bytes(ts._encrypt(hash))

token = (msg + b'|' + mac).hex()

# ============================== LOGIN ============================== 
token = bytes.fromhex(token)
msg, _, mac = token.partition(b'|')
hash = sha256(secret + msg).digest()
w = long_to_bytes(ts._decrypt(bytes_to_long(mac)))
if hash == w[-32:]:
    user = msg.rpartition(b'user=')[2]
    print(f'Welcome {user}!')
    if user == b'admin':
        print(FLAG)


In [ ]:
port = 30010
r = remote('chal.2025.ductf.net', port)

# Lấy n:
n = r.recvline(keepends=False).strip().decode()
print (f"n: {n}")

# Lựa chọn đăng ký
get_success = r.recvuntil('> ').decode()
r.sendline('1')



[x] Opening connection to chal.2025.ductf.net on port 30010
[x] Opening connection to chal.2025.ductf.net on port 30010: Trying 34.40.180.133
[+] Opening connection to chal.2025.ductf.net on port 30010: Done
n: 62193965648465399096623579181477183233254274930926626512971528815428012636286805522802093361594855935320197783306741361574261363816402418047847456810197277141794374405377051042177440809964553006570936846928894075273768966204852427590549910876763841563072715373847778452274966968494291019990427851608556125393


In [5]:
msg = b'13user=admuser=inuik'
user = msg.rpartition(b'user=')[2]
print(user)

b'inuik'
